In [33]:
import requests
import json
import pandas as pd
import numpy as np

In [34]:
token = '61b0de5e-5006-0203-de71-d778b72cc02d'

In [35]:
def url_to_df(url: str, token: str, main_col_name: str):
    """This function receives the INEGI API url, and the token,
    makes the petition and if the response is 200, then json_to_df
    is called and the response json is passed. The df is returned."""
    res = requests.get(url.replace('[Aquí va tu Token]', token))
    if res.status_code == 200:
        df = json_to_df(res.json(), main_col_name)
        return df
    return None

def json_to_df(response_json, main_col_name):
    df = pd.json_normalize(response_json['Series'][0]['OBSERVATIONS'])
    df = df[['TIME_PERIOD', 'OBS_VALUE']].rename(columns={'OBS_VALUE': main_col_name, 'TIME_PERIOD': 'year'}, inplace=False)
    return df

In [36]:
nacimientos_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000026/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
defunciones_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000030/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
matrimonios_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000038/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
divorcios_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000039/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
poblacion_total_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000001/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
homicidios_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6300000264/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
porcentaje_pib_educacion_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6207067825/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
suicidios_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6200240338/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
median_age_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000010/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'

In [37]:
nacimientos = url_to_df(nacimientos_url, token, 'nacimientos')
defunciones = url_to_df(defunciones_url, token, 'defunciones')
matrimonios = url_to_df(matrimonios_url, token, 'matrimonios')
divorcios = url_to_df(divorcios_url, token, 'divorcios')
poblacion_total = url_to_df(poblacion_total_url, token, 'poblacion_total')
porcentaje_pib_educacion = url_to_df(porcentaje_pib_educacion_url, token, 'porcentaje_pib_educacion')
homicidios_total = url_to_df(homicidios_url, token, 'homicidios_total')
suicidios = url_to_df(suicidios_url, token, 'suicidios')
median_age = url_to_df(median_age_url, token, 'median_age')

In [40]:
inegi_df = (
    pd.merge(right=nacimientos ,left=defunciones ,how='inner', left_on='year', right_on='year')
    .merge(right=matrimonios, left_on='year',right_on='year')
    .merge(right=divorcios, left_on='year',right_on='year')
    .merge(right=homicidios_total, left_on='year',right_on='year', how='left')
    .merge(right=suicidios, left_on='year',right_on='year', how='left')
)

In [46]:
inegi_df.replace('.00000000000000000000','',regex=True, inplace=True)

In [142]:
inegi_df.head()

,defunciones,nacimientos,matrimonios,divorcios,homicidios_total,suicidios
year,,,,,,
1994,419074,2904389,671640,35029,15839,2603
1995,430278,2750444,658114,37455,15612,2894
1996,436321,2707718,670523,38545,14505,3020
1997,440437,2698425,707840,40792,13552,3370
1998,444665,2668428,704456,45889,13656,3342


In [113]:
inegi_df.set_index('year', inplace=True)

#### Set de datos del banco mundial

In [117]:
pre = pd.read_csv('datasets/API_MEX_DS2_en_csv_v2_32388.csv', skiprows=[0,1,2,3]).drop(['Country Code', 'Country Name', '2023', 'Unnamed: 68'], axis=1)

In [98]:
pre.head(3)

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"Intentional homicides, female (per 100,000 fem...",VC.IHR.PSRC.FE.P5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.423447,3.972907,3.885846,4.534932,5.469392,5.924644,6.092515,6.143375,6.171259,NaN
1,Battle-related deaths (number of people),VC.BTL.DETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Voice and Accountability: Percentile Rank,VA.PER.RNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.051643,48.768475,44.334976,43.842365,42.857143,45.631069,45.893719,44.927536,43.961353,42.028984


In [99]:
columnas_a_preservar = ['Indicator Name', 'Indicator Code','1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022']

In [118]:
pre = pre[columnas_a_preservar]

In [119]:
indicadores_a_preservar = ['SP.POP.GROW', 'SP.POP.65UP.TO.ZS', 'SP.DYN.TFRT.IN', 'SP.URB.TOTL', 'SP.RUR.TOTL', 'SP.POP.TOTL']

In [120]:
pre = pre[pre['Indicator Code'].isin(indicadores_a_preservar)]

In [122]:
pre.drop('Indicator Code', inplace=True, axis=1)

In [123]:
pre = pre.melt(id_vars=['Indicator Name'], value_vars=['1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022'], value_name='Value')

In [124]:
pre

,Indicator Name,variable,Value
0,Population growth (annual %),1994,1.904435e+00
1,"Fertility rate, total (births per woman)",1994,3.127000e+00
2,Population ages 65 and above (% of total popul...,1994,4.386815e+00
3,Urban population,1994,6.445628e+07
4,Rural population,1994,2.385814e+07
...,...,...,...
169,"Fertility rate, total (births per woman)",2022,1.803000e+00
170,Population ages 65 and above (% of total popul...,2022,8.323138e+00
171,Urban population,2022,1.036609e+08
172,Rural population,2022,2.384327e+07


In [130]:
wb_df = pre.pivot(columns='Indicator Name',index='variable', values='Value')

In [147]:
wb_df.head(2)

,"Fertility rate, total (births per woman)",Population ages 65 and above (% of total population),Population growth (annual %),"Population, total",Rural population,Urban population
variable,,,,,,
1994,3.127,4.386815,1.904435,88314424.0,23858142.0,64456282.0
1995,3.038,4.471475,1.856808,89969572.0,23960696.0,66008876.0


In [151]:
wb_df.rename( 
    columns={
        'Fertility rate, total (births per woman)':'Fertility rate', 
        'Population ages 65 and above (% of total population)':'Population above 65 years',
        'Population, total': 'Total population'
    },
    inplace=True
)

In [152]:
wb_df.index.name = 'year'

In [156]:
#Join the INEGI and World Bank datasets
df_2 = pd.concat([wb_df, inegi_df], axis=1)

In [157]:
df_2.head(2)

,Fertility rate,Population above 65 years,Population growth (annual %),Total population,Rural population,Urban population,defunciones,nacimientos,matrimonios,divorcios,homicidios_total,suicidios
year,,,,,,,,,,,,
1994,3.127,4.386815,1.904435,88314424.0,23858142.0,64456282.0,419074,2904389,671640,35029,15839,2603
1995,3.038,4.471475,1.856808,89969572.0,23960696.0,66008876.0,430278,2750444,658114,37455,15612,2894


In [163]:
#Homogenizar los nombre de las columnas
df_2.rename(
    columns={
        'Fertility rate':'Tasa fertilidad',
        'Population above 65 years':'Población 65 años y más',
        'Population growth (annual %)': 'Crecimiento poblacional (% anual)',
        'Total population': 'Población total',
        'Rural population':'Población rural',
        'Urban population':'Población urbana',
        'defunciones': 'Defunciones',
        'nacimientos':'Nacimientos',
        'matrimonios': 'Matrimonios',
        'divorcios':'Divorcios',
        'homicidios_total': 'Homicidios total',
        'suicidios':'Suicidios'
    },
    inplace=True
)

In [182]:
#Transformar el tipo de datos de las columnas que deberían ser INT
columnas_a_convertir = df_2.loc[:,'Población total':].columns

In [180]:
df_2[columnas_a_convertir] = df_2[columnas_a_convertir].astype(np.int64)

In [184]:
df_2.to_csv('datasets/final_df2.csv', index=True)

#### set de datos INEGI

In [186]:
db = pd.read_excel('datasets/estructura_00.xlsx')

In [187]:
db = db.query("desc_municipio == 'Estatal' ")

In [188]:
indicadores = [
    "Población total",
    "Edad mediana",
    "Población de 0 a 4 años",
    "Población de 5 a 9 años",
    "Población de 10 a 14 años",
    "Población de 15 a 19 años",
    "Población de 20 a 24 años",
    "Población de 25 a 29 años",
    "Población de 30 a 34 años",
    "Población de 35 a 39 años",
    "Población de 40 a 44 años",
    "Población de 45 a 49 años",
    "Población de 50 a 54 años",
    "Población de 55 a 59 años",
    "Población de 60 a 64 años",
    "Población de 65 a 69 años",
    "Población de 70 a 74 años",
    "Población de 75 a 79 años",
    "Población de 80 a 84 años",
    "Población de 85 a 89 años",
    "Población de 90 a 94 años",
    "Población de 95 a 99 años",
    "Población de 100 años y más"
]

In [194]:
df_inegi_pre = db.query('indicador in @indicadores')

In [195]:
df_inegi_pre.info()

<class 'pandas.core.frame.DataFrame'>
Index: 736 entries, 136 to 324996
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   desc_entidad  736 non-null    object 
 1   indicador     736 non-null    object 
 2   1995          736 non-null    object 
 3   2000          736 non-null    object 
 4   2005          736 non-null    float64
 5   2010          736 non-null    float64
 6   2020          736 non-null    float64
dtypes: float64(3), object(4)
memory usage: 46.0+ KB


In [196]:
#Seleccionamos las 'columnas a ocupar por el momento'
#utilizamos lo censos con todos lo datos disponibles
df_inegi_pre = df_inegi_pre.loc[:,['desc_entidad', 'indicador','1995','2000','2005','2010','2020']]

In [201]:
df_inegi_pre.head()

,Estado,Indicador,1995,2000,2005,2010,2020
136,Aguascalientes,Población total,862720,944285,1065416.0,1184996.0,1425607.0
145,Aguascalientes,Edad mediana,19,21,23.0,24.0,27.0
152,Aguascalientes,Población de 0 a 4 años,112429,116137,118536.0,121557.0,124430.0
155,Aguascalientes,Población de 10 a 14 años,104083,108532,120280.0,123806.0,129717.0
158,Aguascalientes,Población de 100 años y más,59,102,98.0,109.0,112.0


In [274]:
df_inegi_pre['Estado'] = np.where(df_inegi_pre['Estado'] == 'Coahuila de Zaragoza', 'Coahuila', df_inegi_pre['Estado'])
df_inegi_pre['Estado'] = np.where(df_inegi_pre['Estado'] == 'Michoacán de Ocampo', 'Michoacán', df_inegi_pre['Estado'])
df_inegi_pre['Estado'] = np.where(df_inegi_pre['Estado'] == 'Ciudad de Mexico', 'Ciudad de México', df_inegi_pre['Estado'])

In [275]:
df_inegi_pre.Estado.unique()

array(['Aguascalientes', 'Baja California', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Ciudad de México', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'México', 'Michoacán', 'Morelos', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz de Ignacio de la Llave', 'Yucatán',
       'Zacatecas'], dtype=object)

In [276]:
df_inegi_pre.rename(columns={'desc_entidad':'Estado','indicador':'Indicador'}, inplace=True)

In [277]:
#Change the df from a wide format to a long format
df_melted = df_inegi_pre.melt(id_vars=['Estado', 'Indicador'], value_vars=['1995', '2000', '2005', '2010', '2020'], var_name='Year', value_name='Value')

In [278]:
#Change the datatype of the Year column
df_melted.Year = df_melted.Year.astype(np.int64)

In [279]:
df_melted.Year

0       1995
1       1995
2       1995
3       1995
4       1995
        ... 
3675    2020
3676    2020
3677    2020
3678    2020
3679    2020
Name: Year, Length: 3680, dtype: int64

In [280]:
#Change the name of Veracruz de Ignacio de la Llave to Veracruz
df_melted['Estado'] = np.where(df_melted['Estado'] == 'Veracruz de Ignacio de la Llave', 'Veracruz', df_melted['Estado'])

In [281]:
df_pivoted = df_melted.pivot(columns='Indicador', values='Value', index=['Estado','Year'])

In [262]:
df_pivoted.loc['Veracruz']

Indicador,Edad mediana,Población de 0 a 4 años,Población de 10 a 14 años,Población de 100 años y más,Población de 15 a 19 años,Población de 20 a 24 años,Población de 25 a 29 años,Población de 30 a 34 años,Población de 35 a 39 años,Población de 40 a 44 años,...,Población de 55 a 59 años,Población de 60 a 64 años,Población de 65 a 69 años,Población de 70 a 74 años,Población de 75 a 79 años,Población de 80 a 84 años,Población de 85 a 89 años,Población de 90 a 94 años,Población de 95 a 99 años,Población total
Year,,,,,,,,,,,,,,,,,,,,,
1995,21,776467,807421,1732,735362,647799,543854,474740,432316,336144,...,183424,154740,112997,82096,51656,32694,19582,8609,4665,6737324
2000,23,728473,797328,2261,709056,593263,536397,498825,459768,380702,...,205722,178582,132984,97101,66604,36758,22336,9608,5467,6908975
2005,25.0,676902.0,791110.0,2115.0,710167.0,582248.0,517828.0,516830.0,496595.0,432692.0,...,247750.0,211512.0,159801.0,121464.0,85434.0,50171.0,27716.0,10668.0,5679.0,7110214.0
2010,27.0,662288.0,741526.0,2186.0,759388.0,642093.0,557762.0,539685.0,542754.0,486810.0,...,298171.0,239698.0,183836.0,155346.0,100119.0,62627.0,35565.0,13213.0,5967.0,7643194.0
2020,31.0,600253.0,685468.0,2157.0,681548.0,600394.0,580421.0,568121.0,558791.0,537667.0,...,408710.0,347850.0,279233.0,203156.0,145737.0,95232.0,54398.0,21825.0,8304.0,8062579.0


In [282]:
#Set the correct data type
df_temp = df_pivoted.astype(np.int64)

#### Let's create a DF with longer ranges of ages and the percentage that it represents for that given year and age range

In [286]:
#0-19 años
df_temp['Porcentaje de 0-19 años']= round((
    (
        (
            df_temp['Población de 0 a 4 años'] + 
            df_temp['Población de 5 a 9 años'] + 
            df_temp['Población de 10 a 14 años'] + 
            df_temp['Población de 15 a 19 años']
        )
    / df_temp['Población total']
    ) * 100
),2)

#20-39 años
df_temp['Porcentaje de 20-39 años']= round((
    (
        (
            df_temp['Población de 20 a 24 años'] + 
            df_temp['Población de 25 a 29 años'] + 
            df_temp['Población de 30 a 34 años'] + 
            df_temp['Población de 35 a 39 años']
        )
    / df_temp['Población total']
    ) * 100
),2)
#40-59 años
df_temp['Porcentaje de 40-59 años']= round((
    (
        (
            df_temp['Población de 40 a 44 años'] + 
            df_temp['Población de 45 a 49 años'] + 
            df_temp['Población de 50 a 54 años'] + 
            df_temp['Población de 55 a 59 años']
        )
    / df_temp['Población total']
    ) * 100
),2)
#60-79 años
df_temp['Porcentaje de 60-79 años']= round((
    (
        (
            df_temp['Población de 60 a 64 años'] + 
            df_temp['Población de 65 a 69 años'] + 
            df_temp['Población de 70 a 74 años'] + 
            df_temp['Población de 75 a 79 años']
        )
    / df_temp['Población total']
    ) * 100
),2)
#+80 años
df_temp['Porcentaje de 80 y más años']= round((
    (
        (
            df_temp['Población de 80 a 84 años'] + 
            df_temp['Población de 85 a 89 años'] + 
            df_temp['Población de 90 a 94 años'] + 
            df_temp['Población de 95 a 99 años'] +
            df_temp['Población de 100 años y más']
        )
    / df_temp['Población total']
    ) * 100
),2)


In [287]:
df_1 = df_temp

In [289]:
df_1

Indicador            Edad mediana  Población de 0 a 4 años  \
Estado         Year                                          
Aguascalientes 1995            19                   112429   
               2000            21                   116137   
               2005            23                   118536   
               2010            24                   121557   
               2020            27                   124430   
...                           ...                      ...   
Zacatecas      1995            20                   160610   
               2000            21                   156356   
               2005            23                   143830   
               2010            25                   152037   
               2020            28                   151745   

Indicador            Población de 10 a 14 años  Población de 100 años y más  \
Estado         Year                                                           
Aguascalientes 1995                     104083                           59   
               2000                     108532                          102   
               2005                     120280                           98   
               2010                     123806                          109   
               2020                     129717                          112   
...                                        ...                          ...   
Zacatecas      1995                     173536                          192   
               2000                     171383                          260   
               2005                     156277                          267   
               2010                     152154                          253   
               2020                     151355                          239   

Indicador            Población de 15 a 19 años  Población de 20 a 24 años  \
Estado         Year                                                         
Aguascalientes 1995                      97061                      88901   
               2000                      99577                      87508   
               2005                     107007                      93139   
               2010                     121428                     106305   
               2020                     131967                     127020   
...                                        ...                        ...   
Zacatecas      1995                     159590                     126861   
               2000                     145464                     119820   
               2005                     145349                     112509   
               2010                     150499                     131079   
               2020                     141208                     130238   

Indicador            Población de 25 a 29 años  Población de 30 a 34 años  \
Estado         Year                                                         
Aguascalientes 1995                      71839                      62217   
               2000                      79200                      70317   
               2005                      83073                      82651   
               2010                      93604                      88726   
               2020                     118426                     106825   
...                                        ...                        ...   
Zacatecas      1995                      98409                      84527   
               2000                     101757                      91476   
               2005                      99137                      97454   
               2010                     107817                     105427   
               2020                     117673                     114442   

Indicador            Población de 35 a 39 años  Población de 40 a 44 años  \
Estado         Year                                                         
Aguasca

In [290]:
df_1.to_csv('datasets/final_df1.csv', index=True)